In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import asyncio

In [3]:
load_dotenv(override=True)

True

In [4]:
async def run_springboot_agent(requirements: str):
    SPRING_TASKS_SCHEMA = r"""
Return ONLY valid JSON:
{
  "backend_plan": {
    "stack": {
      "java": "17|21",
      "spring_boot": "3.x",
      "modules": ["api", "service", "data"]
    },
    "entities": [
      {"name": "string", "fields": [{"name":"string","type":"string","constraints":["nullable|unique|..."]}]}
    ],
    "endpoints": [
      {"method":"GET|POST|PUT|DELETE","path":"/api/...","summary":"string","request":"schema|none","response":"schema"}
    ],
    "config": {
      "persistence": {"db":"postgres","schema":"public","ddl":"update|validate"},
      "security": {"provider":"keycloak|none","roles":["string","..."]}
    }
  },
  "acceptance_tests": ["curl or httpie examples hitting endpoints"],
  "artifacts": ["relative/paths/that/will/exist/after/build"]
}
"""

    instructions_spring_boot_agent = f"""
You are the Spring Boot Agent.
Goal: produce a concrete backend plan (entities, endpoints, configs) compatible with the contract. Analyze the provided requirements from the Project Manager and prepare the necessary estimates.

Inputs:
- backend_contract (from Feature Broker)
- requirements (from Project Manager)

Tasks:
1) Confirm stack (Java 21, Spring Boot 3.x).
2) Specify entities, repositories, services, controllers.
3) Define endpoint specs with request/response skeletons.
4) Security (Keycloak optional hook, roles).
5) Provide runnable test commands (curl/httpie).
6) List artifact paths that will exist after generation.
7) Based on the requirements, include brief effort/estimate notes for each major module (API, service, data) and for security.

Output format:
{SPRING_TASKS_SCHEMA}

Requirements:
{requirements}

Rules:
- Keep DTOs explicit (avoid leaking entities).
- Use HTTP status codes correctly.
- No non-JSON prose.
"""

    sprng_boot_agent = Agent(
        name="Spring Boot TL",
        instructions=instructions_spring_boot_agent,
        model="gpt-4o-mini"
    )
    result = await Runner.run(sprng_boot_agent,requirements)
    return result


In [5]:
async def run_react_agent(requirements: str):
    REACT_TASKS_SCHEMA = r"""
Return ONLY valid JSON:
{
  "frontend_plan": {
    "stack": {"framework":"react","router":"react-router","state":"zustand|redux|context"},
    "routes": [
      {"path":"/","component":"Home","data":"what it needs","actions":["list|create|update|delete"]}
    ],
    "ui": [
      {"component":"FeatureList","props":["features"],"states":["loading","error","empty"]}
    ],
    "api_clients": [
      {"name":"FeatureApi","methods":[{"name":"list","verb":"GET","path":"/api/features"}]}
    ]
  },
  "acceptance_tests": ["playwright/cypress scenario bullets"],
  "artifacts": ["src/App.tsx","src/routes/*.tsx","src/api/*.ts"]
}
"""

    instructions_react_agent = f"""
You are the React Agent.
Goal: produce a concrete frontend plan that consumes the backend API. Analyze the provided requirements from the Project Manager and prepare the necessary estimates.

Inputs:
- frontend_contract (from Feature Broker)
- requirements (from Project Manager)

Tasks:
1) Define routes, components, and API clients.
2) Specify loading/error/empty UI states.
3) Include minimal state management approach.
4) Provide acceptance test scenarios (Cypress/Playwright bullets).
5) List artifact paths to be generated.
6) Based on the requirements, include brief effort/estimate notes for major areas (routing, state, API clients, UI components, testing).

Output format:
{REACT_TASKS_SCHEMA}

Requirements:
{requirements}

Rules:
- No CSS framework assumptions beyond standard React + a router (can be swapped later).
- Keep components small and composable.
- No non-JSON prose.
"""

    react_agent = Agent(
        name="React TL",
        instructions=instructions_react_agent,
        model="gpt-4o-mini"
    )
    result = await Runner.run(react_agent,requirements)
    return result


In [6]:
async def run_qa_agent(requirements: str):
    QA_REPORT_SCHEMA = r"""
Return ONLY valid JSON:
{
  "summary": "string",
  "backend_checks": [{"name":"string","status":"PASS|FAIL","notes":"string"}],
  "frontend_checks": [{"name":"string","status":"PASS|FAIL","notes":"string"}],
  "gaps": ["string","..."],
  "ready_to_ship": true|false
}
"""

    instructions_qa_verifier = f"""
You are the QA/Verifier Agent.
Goal: verify backend + frontend plans align with feature acceptance criteria. Analyze the provided requirements from the Project Manager and prepare the necessary QA verification and estimates.

Inputs:
- features JSON (final)
- backend_plan JSON
- frontend_plan JSON
- requirements (from Project Manager)

Tasks:
1) Cross-check endpoints vs. frontend api_clients (paths/verbs/fields).
2) Validate each feature has test coverage plan (backend or frontend).
3) Identify gaps and blockers.
4) Provide rough QA effort/estimates based on requirements and coverage.

Output format:
{QA_REPORT_SCHEMA}

Requirements:
{requirements}

Rules:
- ready_to_ship is true ONLY if there are no FAIL checks and no critical gaps.
- No non-JSON prose.
"""

    qa_agent = Agent(
        name="QA TL",
        instructions=instructions_qa_verifier,
        model="gpt-4o-mini"
    )
    result = await Runner.run(qa_agent,requirements)
    return result


In [7]:
async def run_project_manager_standalone(idea: str, project_title: str = "Project", model: str = "gpt-4o-mini"):
    REQUIREMENTS_SCHEMA = r"""
Return ONLY valid JSON:
{
  "project_title": "string",
  "idea": "string",
  "executive_summary": "short overview",
  "features": [
    {"name":"string","description":"string","priority":"must|should|could"}
  ],
  "non_functional": {
    "auth":"keycloak|custom|none",
    "logging_observability":"string",
    "error_handling":"string",
    "performance":"string",
    "accessibility":"string",
    "i18n":"string",
    "privacy_security":"string",
    "scalability":"string"
  },
  "system_overview": {
    "backend":"high-level notes",
    "frontend":"high-level notes",
    "data":"high-level notes",
    "integrations":[ "string", "..." ]
  },
  "assumptions":[ "string", "..." ],
  "risks":[ "string", "..." ],
  "open_questions":[ "string", "..." ]
}
"""

    instructions_project_manager = f"""
You are the Project Manager Agent (Standalone).

Mission
- Turn the user's IDEA into a clear, decision-ready REQUIREMENTS DRAFT.
- Do NOT call any tools. Do NOT perform handoffs. Produce requirements only.

Inputs:
- project_title
- idea

Given:
- project_title: {project_title}
- idea: {idea}

Operating procedure (STRICT)
1) Requirements (your own work only; no tools)
   - Write a concise executive summary.
   - List 5–12 features with short descriptions and priorities (must/should/could).
   - Capture cross-cutting needs (auth, logging/observability, error handling, performance, accessibility, i18n, privacy/security, scalability).
   - Provide a high-level system overview (backend, frontend, data, integrations).
   - Record assumptions, key risks, and open questions.

Final Output
- Return ONLY a single JSON object that strictly matches this schema:
{REQUIREMENTS_SCHEMA}

Guardrails
- No non-JSON prose.
- No estimates, no backend/frontend/QA plans.
- No tool calls or handoffs.
"""

    pm_agent = Agent(
        name="Project Manager (Standalone)",
        instructions=instructions_project_manager,
        model=model
    )
    result = await Runner.run(pm_agent,idea)
    return result


In [16]:
async def run_project_estimation_manager(
    idea: str,
    project_title: str = "Project",
    requirements_list: str | None = None,
    model: str = "gpt-4o-mini",
    hours_per_day: int = 8,
    hours_per_point: int = 6
):
    """
    Build a standalone Project Estimation Manager agent that aggregates requirements + estimates
    and returns a single detailed estimation report as STRICT JSON.
    """
    import json

    ESTIMATION_REPORT_SCHEMA = r"""
Return ONLY valid JSON:
{
  "project_title": "string",
  "idea": "string",
  "executive_summary": "string",
  "requirements_overview": {
    "features": [{"name":"string","priority":"must|should|could"}],
    "non_functional": ["string","..."]
  },
  "estimates_input": [
    {
      "area": "backend|frontend|qa|devops|design|pm|security|other",
      "unit": "hours|days|story_points",
      "value": 0,
      "range": {"min": 0, "max": 0},
      "notes": "string"
    }
  ],
  "effort_summary": {
    "normalization_rules": {
      "hours_per_day": 8,
      "hours_per_point": 6
    },
    "unit": "hours",
    "by_area": [
      {
        "area": "backend|frontend|qa|devops|design|pm|security|other",
        "optimistic": 0,
        "most_likely": 0,
        "pessimistic": 0,
        "assumptions": ["string","..."]
      }
    ],
    "total": {
      "optimistic": 0,
      "most_likely": 0,
      "pessimistic": 0
    }
  },
  "critical_path": ["string","..."],
  "sequencing": [
    {"order":1,"workstream":"string","depends_on":["string","..."],"deliverables":["string","..."]}
  ],
  "risks_merged": ["string","..."],
  "assumptions_merged": ["string","..."],
  "gaps": ["string","..."],
  "next_steps": ["string","..."],
  "appendix": {
    "raw_requirements": [],
    "raw_estimates": []
  }
}
"""

    # Safe defaults for empty inputs
    requirements_list = requirements_list or ''

    instructions_project_estimator = f"""
You are the Project Estimation Manager (Standalone).

Mission
- Produce a single, decision-ready PROJECT ESTIMATION REPORT as STRICT JSON.
- Do NOT call tools. Do NOT perform handoffs. Use ONLY the provided inputs below.

Inputs (raw)
- project_title: {project_title}
- idea: {idea}
- requirements_list JSON (array): {requirements_list}

Normalization Rules (STRICT)
- Convert ALL effort to HOURS.
- If an item uses "days", multiply by {hours_per_day} to get hours.
- If an item uses "story_points", multiply by {hours_per_point} to get hours.
- Preserve any provided optimistic/most_likely/pessimistic or range values; convert each to hours using the rules above.
- If ranges are missing, treat "value" as "most_likely"; derive optimistic = round(most_likely * 0.8), pessimistic = round(most_likely * 1.25).
- NEVER fabricate new work items; if something is missing, mark it in "gaps".
- Sum by area and compute totals for optimistic/most_likely/pessimistic.

Operating Procedure
1) Requirements Overview
   - Extract a concise list of features (name + priority) and non-functional themes from requirements_list.
2) Estimates Aggregation
   - Echo inputs into "estimates_input".
   - Normalize to hours and produce "effort_summary.by_area" and "effort_summary.total".
3) Planning Context
   - Identify critical path and a practical high-level sequencing.
4) Risks, Assumptions, Gaps, Next Steps
   - Merge from inputs; do NOT invent specifics beyond what is implied. If unknown, note as a gap.

Final Output
- Return ONLY one JSON object that EXACTLY matches this schema:
{ESTIMATION_REPORT_SCHEMA}

Guardrails
- No non-JSON prose.
- No dates or promises; estimates are effort-only.
- If any essential info is missing, include a clear entry in "gaps".
"""

    estimator_agent = Agent(
        name="Project Estimation Manager (Standalone)",
        instructions=instructions_project_estimator,
        model=model
    )

    # Kick off the agent with a neutral prompt; everything needed is in the instructions.
    result = await Runner.run(estimator_agent, "Generate the consolidated estimation report now.")
    return result


In [12]:
def _as_text(obj) -> str:
    if obj is None:
        return ""
    if isinstance(obj, str):
        return obj
    final_output = getattr(obj, "final_output", None)
    if final_output is not None:
        return final_output
    if isinstance(obj, (dict, list)):
        return json.dumps(obj, ensure_ascii=False)
    return str(obj)

In [14]:
def _as_json(obj):
    txt = _as_text(obj)
    try:
        return json.loads(txt)
    except Exception:
        # if it's not valid JSON, wrap as raw
        return {"raw": txt}

In [20]:
from pprint import PrettyPrinter


idea = """
A tiny web/mobile app for home-tuition businesses to log sessions, auto-calculate charges, and share weekly WhatsApp summaries with parents—no heavy billing system, just clean tracking and UPI-ready payouts.
"""

with trace("Project Estimation trace"):
    print("Starting Requirement analysis...")
    requirements = await run_project_manager_standalone(idea)
    requirements = _as_text(requirements)
    print("Estimating functionalities....")
    tasks = [run_springboot_agent(requirements),run_react_agent(requirements),run_qa_agent(requirements)]
    results = await asyncio.gather(*tasks)
    for result in results:
        print(result.final_output)
    print("Preparing final estimate....")
    requirements_list = [
        requirements,
       results
    ]
    estimate = await run_project_estimation_manager(idea=idea,requirements_list=requirements_list)
    print(estimate.final_output)

Starting Requirement analysis...
Estimating functionalities....
```json
{
  "backend_plan": {
    "stack": {
      "java": "21",
      "spring_boot": "3.x",
      "modules": ["api", "service", "data"]
    },
    "entities": [
      {
        "name": "Session",
        "fields": [
          {"name": "id", "type": "Long", "constraints": ["nullable: false", "unique: true"]},
          {"name": "date", "type": "LocalDateTime", "constraints": ["nullable: false"]},
          {"name": "duration", "type": "Integer", "constraints": ["nullable: false"]},
          {"name": "subject", "type": "String", "constraints": ["nullable: false"]},
          {"name": "tutorId", "type": "Long", "constraints": ["nullable: false"]}
        ]
      },
      {
        "name": "Charge",
        "fields": [
          {"name": "id", "type": "Long", "constraints": ["nullable: false", "unique: true"]},
          {"name": "totalAmount", "type": "Double", "constraints": ["nullable: false"]},
          {"name": "sessio